In [1]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split, cross_val_score
# from tpot.config.classifier_nn import classifier_config_nn

from sklearn.pipeline import make_pipeline
from tpot.config import classifier_config_dict_light
from tpot.config import classifier_config_dict
from sklearn.neighbors import KNeighborsClassifier

from keras import regularizers

import pandas as pd
import numpy as np
import os
import glob

import time

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
pd.options.mode.chained_assignment = None

C:\Users\monicai\AppData\Local\Continuum\anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
C:\Users\monicai\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
personal_config = classifier_config_dict_light
personal_config = classifier_config_dict

personal_config['tpot.builtins.SparseAutoencoder'] = {
    'regularizer' :['l1__10e-5'], #'regularizers.l2(10e-5)', 'regularizers.l1_l2(10e-5, 10e-5)'],
    'encoding_dim': [10, ],
    'activation': ['relu'],
    'optimizer': ['adadelta', "SGD"],
    'loss':['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error',
       'squared_hinge', 'hinge', 'logcosh', 'binary_crossentropy', 'kullback_leibler_divergence', 'cosine_proximity', 'poisson'],
    'epochs':[10],
    'batch_size':[10, 15]
}


# personal_config['tpot.builtins.SimpleAutoencoder'] = {
#     'regularizer' :['regularizers.l1(10e-5)', 'regularizers.l2(10e-5)', 'regularizers.l1_l2(10e-5, 10e-5)'],
#     'encoding_dim': [40],
#     'activation': ['relu'],
#     'optimizer': ['adadelta', "SGD"],
#     'loss':['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error',
#        'squared_hinge', 'hinge', 'logcosh', 'binary_crossentropy', 'kullback_leibler_divergence', 'cosine_proximity', 'poisson'],
#     'epochs':[100],
#     'batch_size':[200]
# }

In [3]:
#one hot encoding function

#function that takes in a features pandas dataframe and turns it into a numpy array 
#with only the categorical variables one-hot encoded (and leaving out one of the features 
#of every one-hot encoded variable as baseline)
#@param:  features is a pandas df of features; threshold number is the number of unique values 
#a variable should have in order to be considered categorical
#@return:  a numpy matrix containing the original dataset but with the categorical variables
#one hot encoded and one of the one hot encoded features per cat variable left out; 
#for example an original feature set of 10 categorical variables with three categories each will 
#be transoformed into a numpy array with 20 dichotomous variables

def one_hot_encode(features, cat_threshold_number = 5):
    num_unique_vals_dict = {}
    feature_names = list(features)
    for feature in feature_names:
        label_encoder = LabelEncoder()
        features.loc[:, feature] = label_encoder.fit_transform(features.loc[:, feature])
        num_unique_vals_dict[feature] = len(label_encoder.classes_)

    features_to_onehot = []
    for feature in num_unique_vals_dict:
        if num_unique_vals_dict[feature] <= cat_threshold_number and num_unique_vals_dict[feature] > 1:
            features_to_onehot = features_to_onehot + [feature]

    #create index array listing indices in orginal feature names array that are present in features_to_onehot
    indices_to_onehot = np.nonzero(np.in1d(feature_names, features_to_onehot))

    onehot_encoder = OneHotEncoder(categorical_features = indices_to_onehot, sparse = False)
    features = onehot_encoder.fit_transform(features)

    idx_to_delete = np.cumsum([0] + list(num_unique_vals_dict.values()))

    idx_to_keep = [i for i in range(features.shape[1]) if i not in idx_to_delete]

    features = features[:, idx_to_keep]

    return features

In [4]:
#read in data
data = pd.read_table("sample_data.txt")
data = data.iloc[:, :11]
dv = data.iloc[:, 10:11]
features = data.iloc[:, :10]

In [5]:
#one hot encode data, then split it into training and validation
X = one_hot_encode(features)
y = dv
y = pd.np.array(y).ravel()
#split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.33, random_state = 42)
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_val = x_val.reshape((len(x_val)), np.prod(x_val.shape[1:]))

C:\Users\monicai\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\monicai\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [6]:
x_train.shape

(670, 20)

In [7]:
y_train.shape

(670,)

In [8]:
from tpot.builtins import SparseAutoencoder

In [9]:
sae = SparseAutoencoder(encoding_dim=40,
                        regularizer = regularizers.l1(10e-5),
                        activation='relu', 
                        optimizer='adadelta', 
                        loss='binary_crossentropy', 
                        epochs=10, batch_size=10)

In [10]:
sae.fit(x_train, y_train)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 502 samples, validate on 168 samples
Epoch 1/10
502/502 [==============================] - ETA: 8s - loss: 0.717 - ETA: 0s - loss: 0.685 - 0s 532us/step - loss: 0.6762 - val_loss: 0.6437
Epoch 2/10
502/502 [==============================] - ETA: 0s - loss: 0.629 - 0s 84us/step - loss: 0.6173 - val_loss: 0.5897
Epoch 3/10
502/502 [==============================] - ETA: 0s - loss: 0.587 - 0s 101us/step - loss: 0.5675 - val_loss: 0.5427
Epoch 4/10
502/502 [==============================] - ETA: 0s - loss: 0.530 - 0s 104us/step - loss: 0.5200 - val_loss: 0.4959
Epoch 5/10
502/502 [==============================] - ETA: 0s - loss: 0.517 - 0s 95us/step - loss: 0.4722 - val_loss: 0.4500
Epoch 6/10
502/502 [==============================] - ETA: 0s - loss: 0.455 - 0s 101us/step - loss: 0.4249 - val_loss: 0.4055
Epoch 7/10
502/502 [==============================] - ET

In [11]:
sae.transform(x_train).shape

(670, 40)

In [12]:
x_train.shape

(670, 20)

In [13]:
tpot = TPOTClassifier(generations=1, config_dict=personal_config,
                        population_size=10, verbosity=3,
                        template = 'SparseAutoencoder-RandomForestClassifier')

In [14]:
time = time.time()
tpot.fit(x_train, y_train)
end = time.time()
print(end - start)

31 operators have been imported by TPOT.


RuntimeError: There was an error in the TPOT optimization process. This could be because the data was not formatted properly, or because data for a regression problem was provided to the TPOTClassifier object. Please make sure you passed the data to TPOT correctly.